# DropaBot RAG Chatbot Workflow

This notebook builds a professional Retrieval-Augmented Generation (RAG) chatbot for delivery analytics using HuggingFace, FAISS, and Mistral API.


1. Import Required Libraries
2. Load and Prepare Delivery Data & FAQs
3. Generate Embeddings with SentenceTransformers
4. Build FAISS Vector Store
5. Query with Mistral LLM API
6. Demonstrate Sample Analytics Queries


In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import requests

In [2]:
# Load Delivery Data & Sample FAQs/Logs
import sys, os
sys.path.append(os.path.abspath("../src"))
from data_loader import load_delivery_data
df = load_delivery_data()
faqs = [
    "Which city has the highest delays?",
    "How many deliveries failed last week?",
    "Show me fastest couriers in Tanzania",
    "What is the average delivery time in Dar es Salaam?",
    "List all anomalies detected yesterday.",
    "Who are the top 3 couriers by speed?"
 ]

In [3]:
# Generate Embeddings with SentenceTransformers
model = SentenceTransformer('all-MiniLM-L6-v2')
faq_embeddings = model.encode(faqs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Build FAISS Vector Store
dimension = faq_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(faq_embeddings)

In [5]:
# Query with Mistral LLM API
def dropabot_query(user_query, mistral_api_key):
    query_embedding = model.encode([user_query])
    D, I = index.search(query_embedding, k=1)
    retrieved_faq = faqs[I[0][0]]
    prompt = f"Context: {retrieved_faq}\nUser: {user_query}\nAnswer: "
    response = requests.post(
        "https://api.mistral.ai/v1/chat/completions",
        headers={"Authorization": f"Bearer {mistral_api_key}"},
        json={
            "model": "mistral-tiny",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.7
        }
    )
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code} - {response.text}"

In [6]:
# Demonstrate Sample DropaBot Queries
mistral_api_key = "5RD23U4Cu0SD5xmGzcVwxkIbYk07j2Cz"
sample_queries = [
    "Which city has the highest delays?",
    "Show me fastest couriers in Tanzania",
    "How many deliveries failed last week?"
 ]
for q in sample_queries:
    print(f"User Query: {q}")
    print("DropaBot Response:", dropabot_query(q, mistral_api_key))
    print("-"*40)

User Query: Which city has the highest delays?
DropaBot Response: The question does not provide a specific context regarding the type of delays being referred to. However, in terms of traffic congestion, cities like Bangkok, Istanbul, Bogota, and Moscow have been ranked as having some of the highest levels of traffic congestion, resulting in significant delays for commuters. In terms of air travel delays, cities with major international airports such as Chicago, London, and New York often have a high number of delayed flights due to factors such as high volume of passengers, weather conditions, and airport infrastructure. To get the most accurate answer, it would be beneficial to have more context about the specific type of delays being referred to.
----------------------------------------
User Query: Show me fastest couriers in Tanzania
DropaBot Response: The question does not provide a specific context regarding the type of delays being referred to. However, in terms of traffic conge

# Productionizing DropaBot RAG Chatbot

- Deploy the chatbot as a REST API using FastAPI or Flask
- Secure the Mistral API key using environment variables
- Add logging and error handling for robustness
- Scale FAISS index for large datasets
- Integrate with frontend (React) for user interaction
- Monitor usage and performance
